# scRNAseq
 Transcript levels (CPM+1) per transcript-positive cell is shown for the same classes of inducible genes at 0, 2 and 8h after LPS activation. Boxes show upper and lower quartiles and whiskers show 1.5 of the interquartile range. Numbers represent adjusted P-values for 0h versus 2h and 0h versus 8h. Wilcoxon signed rank test with Bonferroni correction for multiple testing. One experiment with 682 cells.

In [ ]:
ENV["Code"] = "../../Code"
for folder in readdir(ENV["Code"]); push!(LOAD_PATH, normpath(ENV["Code"], folder));end
using SingleCellExperiment
using GeneSets
using Seaborn
using PyPlot
using PyCall
using Statistics
using NoLongerProblems_Pandas
import Pandas
using MultipleTesting
using HypothesisTests 
using DataFrames
using CSV
using ProgressMeter
using PrettyPlotting
using ScikitLearn
@sk_import linear_model: LogisticRegression

include("../Databases/Cuartero2018.jl")
include("../Databases/Bhatt2012.jl")
include("../Databases/MF_SingleCell.jl")
include("../Code/Load_scRNAseqData.jl")

sce = SingleCellExperiment.get_cells_with_this_characteristics(["WT"], :Genotype, sce)
sce = SingleCellExperiment.fit_mu_std_alpha(sce, splitdataby = :Sample, assay = "lnCPMplus1")
sce = SingleCellExperiment.fit_single_cell_logistic_regression(sce, assay = "lnCPMplus1")
bhattgenes = Bhatt2012.inducible_genes_figure3()[!,:GeneSymbol];
sce = SingleCellExperiment.Shalek2014_module_score(collect(bhattgenes), sce,fitparameter = "mu", modulescore_name = :BhattGenesScore, untreated_pattern = "UT",comparedtothissample = "WT", assay = "CPM")
bhattgenes= DataFrames.DataFrame(
    "GeneID" => Bhatt2012.inducible_genes_figure3()[!,:GeneSymbol], 
    "Class"=>Bhatt2012.inducible_genes_figure3()[!,:Class]
    );
sceBhatt = innerjoin(sce.rowData, bhattgenes,on = :GeneID)

col_colors_dict = Dict(
    "WT_UT" => "#f0f0f0",
    "WT_2H" => "#bdbdbd",
    "WT_8H" => "#636363",
    "RAD21_UT" => "#fee0d2",
    "RAD21_2H" => "#fc9272",
    "RAD21_8H" => "#de2d26",
    "RAD21" => "red",
    "WT" => "darkgrey",
    "0" => "#deebf7",
    "2" => "#9ecae1",
    "8" => "#3182bd",
    ) 


In [ ]:
alpha = 0.2
include("adjust_alpha.jl")
include("figureclass.jl")
CSV.write("../SourceData/Figure1f_boxplot.csv",sceBhatt[!, ["GeneID",
 "TotalCounts",
 "TotalCells",
 "FractionCells",
 "WT_UT__mu"
 "WT_UT__alpha"
 "WT_2H__mu"
 "WT_2H__alpha"
 "WT_8H__mu"
 "WT_8H__alpha"
 "Class"]])

In [ ]:
import Pkg
Pkg.status()